The purpose of this notebook is to load the data into two formats:

1. Full dataset (excluding rows with missing data, such as exit dates). This is the dataframe called `df`.
    1. Each person's information gets combined into one row per project enrollment.
2. Each person summarized as one row. This is the dataframe called `df_features`.

You can examine how each sheet is loaded using the dataset loading script: `datasci-sf-homeless-project/src/data/dataset.py`

The script assumes you have access to the data via Dropbox, as mentioned in the sfbrigade Slack team #datasci-homeless channel. Everyone has read access, but talk to Matt, Catherine, or Annalie if you want to be added to the shared folder. If you download the data and/or want to keep it somewhere else, just supply each `process_data_*` function with a datadir argument, e.g.:

```python
df_client = ds.process_data_client(datadir='/path/to/raw/csv/files/')
```

Notes:

- One person can have multiple rows in `df`.
- One person can be enrolled in multiple projects at the same time.
- This notebook does not yet make use of the `Income`, `Service`, or `Bed Inventory` sheets.
- If you save out the CSV at the end (or have it from Dropbox), you can simply load the dataset with the commands:

```python
filename = os.path.join(os.getenv('HOME'), 'Dropbox', 'C4SF-datasci-homeless', 'processed', 'homeless_row_per_enrollment.csv')
df = pd.read_csv(filename, header=0, index_col=0, parse_dates=['Entry Date', 'Exit Date', 'Residential Move In Date'])
```

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
%load_ext autoreload
# # the "1" means: always reload modules marked with "%aimport"
%autoreload 2

from __future__ import absolute_import, division, print_function
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
# from tqdm import tqdm
# import warnings

sns.set_context("poster", font_scale=1.3)
pd.set_option('display.max_columns', 100)

# add the data functions to the path
src_data_dir = os.path.join(os.getcwd(), os.pardir, 'src/data')
sys.path.append(src_data_dir)

# functions to load the data
import dataset as ds

In [2]:
# load in and process the data in separate sheets

df_client = ds.process_data_client()

df_enroll = ds.process_data_enrollment()
# Only keep rows with entry dates starting in 2012
df_enroll = df_enroll[df_enroll['Entry Date'] >= '2012']
# Only keep rows with exit dates before 2016-06-01
df_enroll = df_enroll[df_enroll['Exit Date'] <= '2016-06-01']

df_disability = ds.process_data_disability()

df_healthins = ds.process_data_healthins()

df_benefit = ds.process_data_benefit()

df_income = ds.process_data_income()

df_project = ds.process_data_project()

df_service = ds.process_data_service()

df_bedinv = ds.process_data_bedinventory()

In [3]:
# Join the client information with enrollment information.
# Inner join because we want to only keep individuals
# for whom we have both client and enrollment information.
df = df_client.merge(df_enroll, how='inner', left_index=True, right_index=True)

# just choose the first non-cash benefit; this is too simple!
# TODO: join on the exact Project ID, and possible Date
df = df.merge(df_benefit.reset_index().groupby(by=['Personal ID'])[['Non-Cash Benefit']].nth(0),
              how='left', left_index=True, right_index=True)
# # possible fix for above, but this isn't working properly (results in too many rows);
# # probably need date too, but they do not align
# df.reset_index().merge(df_benefit.reset_index()[['Personal ID', 'Project Entry ID', 'Non-Cash Benefit']].drop_duplicates(),
#                        how='left',
#                        on=['Personal ID', 'Project Entry ID'],
#                       ).drop_duplicates().set_index('Personal ID')

df['Non-Cash Benefit'] = df['Non-Cash Benefit'].fillna('None')

# add information about their disability status
# just choose the first disability; this is too simple!
# TODO: join on the exact Project ID
df = df.merge(df_disability.reset_index().groupby(by=['Personal ID'])[['Disability Type']].nth(0),
              how='left', left_index=True, right_index=True)
# # possible fix for above, but this isn't working properly (results in too many rows);
# # probably need date too, but they do not align
# df.reset_index().merge(df_disability.reset_index()[['Personal ID', 'Project Entry ID', 'Disability Type']].drop_duplicates(),
#                        how='left',
#                        on=['Personal ID', 'Project Entry ID'],
#                       ).drop_duplicates().set_index('Personal ID')

df['Disability Type'] = df['Disability Type'].fillna('None')

# add Project Type Code to DataFrame
df = df.merge(df_project[['Project Name',
                          'Project Type Code',
                          'Address City',
                          'Address Postal Code',
                         ]], left_on=['Project ID'], right_index=True)

# sort by entry date
df = df.sort_values('Entry Date')

In [4]:
# rename the columns to have no spaces
df = ds.rename_columns(df)

In [5]:
# set up to count the number of times a person was in the system
df['enrollments'] = 1

# create feature vectors for each person by subselecting or aggregating their enrollments;
# one row per person
agg = {
    'in_permanent_housing': 'last',
    'enrollments': 'sum',
    'race': 'first',
    'ethnicity': 'first',
    'gender': 'first',
    'veteran_status': 'max',
    'client_age_at_entry': 'last',
    'days_enrolled': 'sum',
    'domestic_violence_victim': 'max',
    'disability_type': 'last',
    'noncash_benefit': 'last',
    'housing_status_project_start': 'last',
    'living_situation_before_program_entry': 'last',
    'continuously_homeless_one_year': 'max',
    'chronic_homeless': 'max',
    'project_name': 'last',
    'project_type_code': 'last',
    }
df_features = df.reset_index().groupby(by=['Personal ID']).agg(agg)

# convert booleans to integers
cols = [
    'domestic_violence_victim',
    'veteran_status',
    'in_permanent_housing',
    'continuously_homeless_one_year',
    'chronic_homeless',
    ]
for col in cols:
    df_features[col] = df_features[col].astype(int)

In [6]:
# number of people in the dataset
df_features.shape

(11362, 17)

In [7]:
# glance at the data
df_features.head()

,project_name,ethnicity,veteran_status,noncash_benefit,continuously_homeless_one_year,housing_status_project_start,in_permanent_housing,domestic_violence_victim,gender,project_type_code,chronic_homeless,client_age_at_entry,race,living_situation_before_program_entry,enrollments,days_enrolled,disability_type
Personal ID,,,,,,,,,,,,,,,,,
173781,MOSBE SOP - Natividad Shelter,Hispanic/Latino,0,Food Stamps,0,Category 1 - Homeless,0,1,Female,Emergency Shelter,0,35,White,"Emergency shelter, including hotel or motel pa...",2,147,None
173782,MOSBE SOP - Natividad Shelter,Hispanic/Latino,0,None,1,Category 1 - Homeless,0,1,Male,Emergency Shelter,0,10,White,"Emergency shelter, including hotel or motel pa...",1,147,None
173783,MOSBE SOP - Natividad Shelter,Hispanic/Latino,0,None,1,Category 1 - Homeless,0,1,Female,Emergency Shelter,0,12,White,"Emergency shelter, including hotel or motel pa...",1,147,None
173803,MOSBE SOP - Natividad Shelter,Hispanic/Latino,0,Food Stamps,0,Category 1 - Homeless,0,1,Female,Emergency Shelter,0,32,White,"Staying or living in a friend's room, apartmen...",1,78,None
173804,MOSBE SOP - Natividad Shelter,Hispanic/Latino,0,None,0,Category 1 - Homeless,0,0,Female,Emergency Shelter,0,11,White,"Staying or living in a friend's room, apartmen...",1,78,None


In [8]:
# save it for easy loading
filename = os.path.join(os.getenv('HOME'), 'Dropbox', 'C4SF-datasci-homeless', 'processed', 'homeless_row_per_enrollment.csv')
df.to_csv(filename)